# Data cleaning

### Loading data file and cleaning blanks

In [50]:
import pandas as pd
import numpy as np

#### Reading each file as a pandas dataframe

In [13]:
df_bom = pd.read_csv('data/zippedData/bom.movie_gross.csv.gz')
df_iname = pd.read_csv('data/zippedData/imdb.name.basics.csv.gz')
df_iakas = pd.read_csv('data/zippedData/imdb.title.akas.csv.gz')
df_ititlebas = pd.read_csv('data/zippedData/imdb.title.basics.csv.gz')
df_ititlecrew = pd.read_csv('data/zippedData/imdb.title.crew.csv.gz')
df_ittilepri = pd.read_csv('data/zippedData/imdb.title.principals.csv.gz')
df_ititlerate = pd.read_csv('data/zippedData/imdb.title.ratings.csv.gz')
df_ireview = pd.read_csv('data/zippedData/rotten_tomatoes_critic_reviews.csv.gz')
df_imov = pd.read_csv('data/zippedData/rotten_tomatoes_movies.csv.gz')
df_rtmovr = pd.read_csv('data/zippedData/tmdb.movies.csv.gz')
df_rt_budget = pd.read_csv('data/zippedData/tn.movie_budgets.csv.gz')

In [14]:
df_bom.sort_values('studio', ascending = True)

,title,studio,domestic_gross,foreign_gross,year
155,Sea Rex 3D: Journey to a Prehistoric World,3D,6100000.0,9900000,2010
1457,Red Obsession,A23,13200.0,NaN,2013
670,Revenge of the Electric Car,A23,151000.0,NaN,2011
1693,Obvious Child,A24,3100000.0,NaN,2014
1717,Tusk,A24,1800000.0,NaN,2014
...,...,...,...,...,...
210,Outside the Law (Hors-la-loi),NaN,96900.0,3300000,2010
555,Fireflies in the Garden,NaN,70600.0,3300000,2011
933,Keith Lemon: The Film,NaN,NaN,4000000,2012
1862,Plot for Peace,NaN,7100.0,NaN,2014


In [20]:
df_studio_gross = df_bom.copy(deep = True)

df_studio_gross.drop(['title', 'year'], axis=1, inplace = True)

df_studio_gross.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 3 columns):
studio            3382 non-null object
domestic_gross    3359 non-null float64
foreign_gross     2037 non-null object
dtypes: float64(1), object(2)
memory usage: 79.5+ KB


#### drop null values and change domestic_gross to int

In [39]:
df_studio_gross['foreign_gross'].isnull().sum()
df_studio_gross['foreign_gross'].fillna(value = 0,inplace = True)
df_studio_gross['foreign_gross'].isnull().sum()
#df_studio_gross.head()
#df_studio_gross.foreign_gross
df_studio_gross['domestic_gross'].isnull().sum()
df_studio_gross['domestic_gross'].dropna(inplace = True)
df_studio_gross['domestic_gross'].isnull().sum()
df_studio_gross.domestic_gross

df_studio_gross.foreign_gross = df_studio_gross.foreign_gross.astype(float)
#df_studio_gross['foreign_gross'] = pd.to_numeric(df_studio_gross['foreign_gross'],errors='coerce')
#df_studio_gross.info()
df_studio_gross.foreign_gross.value_counts()


df_studio_gross['total_gross'] = df_studio_gross['domestic_gross'] + df_studio_gross['foreign_gross']

df_studio_gross.info()
df_studio_gross.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 4 columns):
studio            3382 non-null object
domestic_gross    3359 non-null float64
foreign_gross     3387 non-null float64
total_gross       3359 non-null float64
dtypes: float64(3), object(1)
memory usage: 106.0+ KB


,studio,domestic_gross,foreign_gross,total_gross
0,BV,415000000.0,652000000.0,1.067000e+09
1,BV,334200000.0,691300000.0,1.025500e+09
2,WB,296000000.0,664300000.0,9.603000e+08
3,WB,292600000.0,535700000.0,8.283000e+08
4,P/DW,238700000.0,513900000.0,7.526000e+08


#### Save a copy of cleaned gross data

In [52]:
df_studio_gross.to_csv('data/zippedData/studio_gross.csv', index = False)

df_working = df_rt_budget.copy(deep=True)
#making a copy of the dataset

df_working.head()


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


#### remove ccy from monetary columns and convert columns to numbers

In [53]:
df_working[df_working.columns[3:]] = df_working[df_working.columns[3:]].apply(lambda x: x.str.replace('$','')).apply(lambda x: x.str.replace(',','')).astype(np.int64)

In [55]:
df_working.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
id                   5782 non-null int64
release_date         5782 non-null object
movie                5782 non-null object
production_budget    5782 non-null int64
domestic_gross       5782 non-null int64
worldwide_gross      5782 non-null int64
dtypes: int64(4), object(2)
memory usage: 271.2+ KB


#### adding column to determine foreign gross

In [56]:
df_working['foreign_gross'] = df_working['worldwide_gross'] - df_working['domestic_gross']
df_working.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,foreign_gross
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,2015837654
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,804600000
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350,107000000
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963,944008095
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,696540365


In [57]:
df_working['profit_loss'] = df_working['worldwide_gross'] - df_working['production_budget']
df_working.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,foreign_gross,profit_loss
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,2015837654,2351345279
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,804600000,635063875
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350,107000000,-200237650
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963,944008095,1072413963
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,696540365,999721747


In [67]:
df_working['profitable'] = df_working['profit_loss'].apply(lambda x: 'True' if x > 0 else 'False')

df_working.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,foreign_gross,profit_loss,profitable
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,2015837654,2351345279,True
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,804600000,635063875,True
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350,107000000,-200237650,False
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963,944008095,1072413963,True
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,696540365,999721747,True


#### saving cleaned dataframe to file 

In [69]:
df_working.to_csv('data/zippedData/tn.movie_budget_working.csv')

#### merging basic and rating data frames on tconst

In [71]:
df_merged = pd.merge(df_ititlerate,df_ititlebas, on='tconst')
df_merged.head()

,tconst,averagerating,numvotes,primary_title,original_title,start_year,runtime_minutes,genres
0,tt10356526,8.3,31,Laiye Je Yaarian,Laiye Je Yaarian,2019,117.0,Romance
1,tt10384606,8.9,559,Borderless,Borderless,2019,87.0,Documentary
2,tt1042974,6.4,20,Just Inès,Just Inès,2010,90.0,Drama
3,tt1043726,4.2,50352,The Legend of Hercules,The Legend of Hercules,2014,99.0,"Action,Adventure,Fantasy"
4,tt1060240,6.5,21,Até Onde?,Até Onde?,2011,73.0,"Mystery,Thriller"
